In [2]:
from collections import defaultdict, namedtuple
import numpy as np
import pandas as pd
import math
import random

In [3]:
import duolingo_replica as dr
import duolingo_original as do

We load the data and select a subset of 5000 observations to test the models.

In [5]:
data   = pd.read_csv('https://www.dropbox.com/scl/fi/pnxa2jv4xf23bfwry1q9x/learning_traces.13m.csv?rlkey=2dt9848lutbgyys5sujq8dgw2&dl=1' )
#subset = data.sample( n = 5000, random_state = 5 )
subset = data[data["learning_language"] == "en"]


In [9]:
subset = data[data["learning_language"] == "en"]
subset

,p_recall,timestamp,delta,user_id,learning_language,ui_language,lexeme_id,lexeme_string,history_seen,history_correct,session_seen,session_correct
87,1.000000,1362082504,357,u:dwbJ,en,pt,1052c3ace653dbc8923eaa183bc02b88,definition/definition<n><sg>,17,17,2,2
88,1.000000,1362082504,357,u:dwbJ,en,pt,9cba1b30f88bf3c047b22cffcaf88c12,surface/surface<n><sg>,19,19,3,3
89,1.000000,1362082504,357,u:dwbJ,en,pt,961cd149f20f2571419b1412d849f19a,scale/scale<n><sg>,21,20,3,3
90,0.800000,1362082504,357,u:dwbJ,en,pt,5cbb1249562e95794a4c4ae0e2d8ae26,temperature/temperature<n><sg>,44,36,5,4
91,1.000000,1362082504,357,u:dwbJ,en,pt,2df65bdf80d10d2b78d62cb2e0a731d8,distance/distance<n><sg>,21,20,3,3
...,...,...,...,...,...,...,...,...,...,...,...,...
12854221,0.800000,1363104897,368,u:i5D8,en,it,d5efc552aaea3109eb5388aa1ec8673d,the/the<det><def><sp>,6,4,5,4
12854222,0.800000,1363104897,368,u:i5D8,en,it,a826c47947d68549fa81e19cafa57ba0,eat/eat<vblex><pres>,4,4,5,4
12854223,1.000000,1363104897,368,u:i5D8,en,it,5e29d77697d23070a1fb92eb6c90e9b6,bread/bread<n><sg>,4,4,4,4
12854224,0.600000,1363104897,368,u:i5D8,en,it,cdfecc9247566d40bb964a218c54c783,drink/drink<vblex><pres>,3,2,5,3


# 1. LOGIT

## 1.1. Replication

We perform the train-test split and, in addition, obtain the list of predictor variables. The predictor variables include: 'right,' 'wrong,' 'bias,', 'time' and dummy variables for lexemes.

In [10]:
trainset, testset, feature_vars = dr.read_data( subset, method = 'lr', omit_lexemes = False )

c:\Users\Alexander\Documents\GitHub\NNLangRecall\src\duolingo_replica.py:296: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[ 'p' ]          = df[ 'p_recall' ].apply( lambda x: np.clip( float( x ),  0.0001, .9999 ) )
c:\Users\Alexander\Documents\GitHub\NNLangRecall\src\duolingo_replica.py:297: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[ 't' ]          = df[ 'delta' ].apply( lambda x: float( x ) / ( 60 * 60 * 24 ) )
c:\Users\Alexander\Documents\GitHub\NNLangRecall\src\duolingo_replica.py:298: Setting

MemoryError: Unable to allocate 111. GiB for an array with shape (2983, 5014791) and data type float64

We define and fit a Logistic Regression Model for the replication code

In [5]:
%%time

model_1 = dr.logit_model( feature_columns = feature_vars )
model_1.train( trainset )

CPU times: total: 34.7 s
Wall time: 35.2 s


We evaluate the model with the test set and obtain results. The h_seed allows replicating the values of h_hat, which are random for logistic regression

In [6]:
model_1.test_model( testset, h_seed = 5 )

-----------------------------
            Results          
-----------------------------
Total Loss : 188763.186
p          : 65.696
h          : 188697.488
l2         : 0.002
mae (p)    : 0.332
cor (p)    : -0.022
mae (h)    : 149.518
cor (h)    : -0.024
-----------------------------


We export theta values for the Logit replication model

In [7]:
model_1.dump_theta( 'logit_replication_thetas.txt' )

## 1.2. Original code

We perform the train-test split and, in addition, obtain the list of predictor variables. The predictor variables include: 'right,' 'wrong,' 'bias,' and dummy variables for lexemes.

In [8]:
trainset2, testset2 = do.read_data( subset, method = 'lr' )

done!


We define and fit a Logistic Regression Model for the original code

In [9]:
%%time

model_2 = do.SpacedRepetitionModel( method = 'lr' )
model_2.train( trainset2 )

CPU times: total: 46.9 ms
Wall time: 44.9 ms


We evaluate the model with the test set and obtain results. The first value on the right corresponds to the metric 'Total Loss.' The other metrics are named accordingly. The metrics are almost exactly the same as in the replication code.

In [10]:
model_2.eval( testset2, h_seed = 5 )

188763.117 (p=65.626, h=188697.488, l2=0.002)	mae(p)=0.332	cor(p)=-0.022	mae(h)=149.518	cor(h)=-0.024


We export theta values for the Logit original model. Thetas are almost exactly the same as in the replication code.

In [11]:
model_2.dump_weights( 'logit_original_thetas.txt' )

# 2. HLR Model

## 2.1. Replication

We perform the train-test split and, in addition, obtain the list of predictor variables. The predictor variables include: 'right,' 'wrong,' 'bias,' and dummy variables for lexemes. Unlike Model 1, in this case, the variable 'time' is not included as a predictor.

In [12]:
trainset3, testset3, feature_vars3 = dr.read_data( subset, method = 'hlr', omit_lexemes = False )

We train the HLR replication model

In [13]:
%%time

model_3 = dr.HLR_model( feature_columns = feature_vars3, omit_h_term = True )
model_3.train( trainset3 )

CPU times: total: 42 s
Wall time: 42.4 s


We evaluate the model with the test set and obtain results

In [14]:
model_3.test_model( testset3 )

-----------------------------
            Results          
-----------------------------
Total Loss : 188049.037
p          : 180.259
h          : 187868.778
l2         : 0.000
mae (p)    : 0.431
cor (p)    : 0.011
mae (h)    : 149.080
cor (h)    : -0.092
-----------------------------


We export theta values for the Logit replication model

In [15]:
model_3.dump_theta( 'hlr_replication_thetas.txt' )

## 2.2. Original

We perform the train-test split and, in addition, obtain the list of predictor variables. The predictor variables include: 'right,' 'wrong,' 'bias,' and dummy variables for lexemes. Unlike Model 3, in this case, the variable 'time' is not included as a predictor.

In [16]:
trainset2, testset2 = do.read_data( subset, method = 'hlr' )

done!


We fit the HLR original model. We omit h_term.

In [17]:
%%time

model_4 = do.SpacedRepetitionModel( method = 'hlr', omit_h_term = True )
model_4.train( trainset2 )

CPU times: total: 46.9 ms
Wall time: 53.4 ms


We evaluate the model with the test set and obtain results. The first value on the right corresponds to the metric 'Total Loss.' The other metrics are named accordingly. The metrics are almost exactly the same as in the replication code.

In [18]:
model_4.eval( testset2, h_seed = 5 )

188048.806 (p=180.252, h=187868.554, l2=0.000)	mae(p)=0.431	cor(p)=0.011	mae(h)=149.080	cor(h)=-0.092


We export theta values for the HLR original model. Thetas are almost exactly the same as in the replication code.

In [19]:
model_4.dump_weights( 'hlr_original_thetas.txt' )